# LoRA Streamlit UI: Quickstart for Google Colab
This notebook will help you set up and launch the LoRA Streamlit UI in Google Colab, providing a public link to your app (like Gradio does).

In [ ]:
# 1. Clone your repo (published repo URL)
!git clone https://github.com/melbinjp/Lora_trainer.git
%cd Lora_trainer

In [ ]:
# 2. Ensure setup.sh is executable and run it
!chmod +x setup.sh
!bash setup.sh

In [ ]:
# 3. Install Python dependencies (if not already installed)
!pip install -r requirements.txt

In [ ]:
# 4. Install and set up ngrok for public access
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8080)
print(f"Streamlit public URL: {public_url}")

In [ ]:
# 5. Start Streamlit in the background
!streamlit run test.py --server.port=8080 --server.enableCORS=false --server.enableXsrfProtection=false &